**Практическое задание к уроку 4**

In [87]:
from pprint import pprint
import requests
from lxml import html
from datetime import datetime
from pymongo import MongoClient

In [88]:
# Удаляем лишние символы из текста новости
def clear_text(txt):
    txt = txt.replace('\r', '').replace('\n', '').replace('\xa0', '')
    
    return txt

In [89]:
# Переходим по ссылке каждой новости, чтобы собрать информацию об источнике и дате публикации
def get_source_date(headers, link):
    resp = requests.get(link, headers=headers)
    dom = html.fromstring(resp.text)
    datepub = dom.xpath('//span[contains(@class,"breadcrumbs__item")]//*[@datetime]/@datetime')[0]
    format_str = '%Y-%m-%dT%H:%M:%S%z'
    datepub = datetime.strptime(str(datepub), format_str).strftime("%d-%m-%Y %H:%M")
    source = dom.xpath('//span[contains(@class,"breadcrumbs__item")]//*[@href]/@href')[0]
    
    return source, datepub

In [95]:
client = MongoClient('localhost', 27017)
db = client['news_database']
news_coll = db.news

url = 'https://news.mail.ru'
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36'}

response = requests.get(url, headers=headers)
dom = html.fromstring(response.text)

fishing_items = dom.xpath("//li[contains(@class,'list__item')]")
fl = []
for item in fishing_items:
    fish = {}
    name = item.xpath(".//*[@class='list__text']//text()")[0]
    link = item.xpath(".//*[@class='list__text']//@href")[0]
    source, datepub = get_source_date(headers, link)

    fish['name'] = clear_text(name)
    fish['link'] = link
    fish['source'] = source
    fish['datepub'] = datepub
    
    # Записываем новость в базу, только если такой ссылки ещё нет и наполняем список fl, для проверки
    if not news_coll.find_one({'link': link}):
        news_coll.insert_one(fish)
        fl.append(fish)

for news in news_coll.find({}):
    pprint(news)

{'_id': ObjectId('621ed3ed1246c7fec0ee6b37'),
 'datepub': '01-03-2022 13:54',
 'link': 'https://news.mail.ru/economics/50252360/',
 'name': 'Россиянам назвали самые выгодные способы сбережения денег',
 'source': 'http://iz.ru/'}
{'_id': ObjectId('621ed3ee1246c7fec0ee6b38'),
 'datepub': '01-03-2022 15:32',
 'link': 'https://news.mail.ru/politics/50261817/',
 'name': 'Песков назвал «гиперэмоциональной» реакцию россиян насанкции',
 'source': 'https://www.vedomosti.ru/'}
{'_id': ObjectId('621ed3ef1246c7fec0ee6b39'),
 'datepub': '02-03-2022 03:33',
 'link': 'https://news.mail.ru/politics/50270123/',
 'name': 'Турция отказалась от санкций против России',
 'source': 'http://www.kommersant.ru'}
{'_id': ObjectId('621ed3f01246c7fec0ee6b3a'),
 'datepub': '02-03-2022 01:26',
 'link': 'https://news.mail.ru/economics/50269544/',
 'name': 'Европа начала отказывать России вимпорте иэкспорте',
 'source': 'http://iz.ru/'}
{'_id': ObjectId('621ed3f11246c7fec0ee6b3b'),
 'datepub': '01-03-2022 15:39',
 'li

In [94]:
# Количество новостей
len(fl)

20